In [ ]:
# 2023-10-09 T5

In [ ]:
pip install pinecone-client openai tiktoken langchain unidecode

In [ ]:
import pandas as pd

csv_path = "Replace your path here"
pinecone_index = 'Replace your pinecone index here'

# Load the CSV file
df = pd.read_csv(csv_path)

In [ ]:
import openai
import pinecone
import os
import getpass

# Initialize OpenAI
os.environ["openai.api_key"] = getpass.getpass("OpenAI API Key:")

# Initialize Pinecone
os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")
os.environ["PINECONE_ENV"] = getpass.getpass("Pinecone Environment:")

pinecone.init(
	api_key=os.getenv("PINECONE_API_KEY"),
	environment=os.getenv("PINECONE_ENV")
)

In [ ]:
# Set the model to use
MODEL = "text-embedding-ada-002"

# Create a list to hold the embeddings
embeddings = []

# Batch size for uploading, pinecone recommended around 100 vector / less than 2MB
batch_size = 75

# Loop through the DataFrame in chunks
for i in range(0, len(df), batch_size):
    # Get the batch of strings
    batch = df.iloc[i:i+batch_size, 1].tolist()

    # Create the embeddings
    res = openai.Embedding.create(input=batch, engine=MODEL)

    # Extract the embeddings and add them to the list
    embeddings.extend([record['embedding'] for record in res['data']])

In [ ]:
   # Connect to the index
index = pinecone.Index(pinecone_index)
print(len(df))

3721


In [ ]:
# Define a helper function to break the list into chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


# Import the necessary library
from unidecode import unidecode
import secrets

# Create a list to hold the IDs, embeddings, and metadata
ids_embeddings_metadata = []

# Loop through the DataFrame
for i, row in df.iterrows():

    # Generate a 6-digit random hexadecimal number
    random_hex = secrets.token_hex(3)

    # Convert the ID to ASCII
    ascii_id = unidecode(row[0])
    # Make sure all the ID are unique, the job title will be parse out by removing the first 7 varchar
    unique_id = random_hex + '-' + ascii_id

    # Create a dictionary to hold the ID, embedding, and metadata
    id_embedding_metadata = {
        'id': unique_id,
        'values': embeddings[i],
        'metadata': {
            'row_count': i,
            'csv_column_2': row[1],
            'csv_column_3': row[2],
            'csv_column_4': row[3],
            'csv_column_5': row[4],
            'file_path': csv_path
        }
    }

    # Add the dictionary to the list
    ids_embeddings_metadata.append(id_embedding_metadata)

# Upsert the vectors with their metadata in chunks
for chunk in chunks(ids_embeddings_metadata, batch_size):
    index.upsert(vectors=chunk)
